In [ ]:
import pandas as pd
import numpy as np
import re

GENRE_COL = "Genres / Categories Represented"


In [ ]:
from google.colab import files
uploaded = files.upload()

filename = list(uploaded.keys())[0]
filename

Saving Agents_and_Genres_Detailed.xlsx - Sheet1.csv to Agents_and_Genres_Detailed.xlsx - Sheet1 (1).csv


'Agents_and_Genres_Detailed.xlsx - Sheet1 (1).csv'

In [ ]:
if filename.endswith(".xlsx"):
    df = pd.read_excel(filename)
else:
    df = pd.read_csv(filename)
df.head(3)


,Agent Name,Agency,Country,Genres / Categories Represented,Sources (URLs only),Notable Clients / Titles
0,Jennifer Laughran,Andrea Brown Literary Agency,USA,Children’s specialist: middle grade (contempor...,https://andreabrownlit.com/agents/jennifer-lau...,"Stephanie Perkins (ANNA AND THE FRENCH KISS), ..."
1,Jennifer March Soloway,Andrea Brown Literary Agency,USA,Children’s/YA across categories; loves high-st...,https://andreabrownlit.com/agents/jennifer-mar...,NaN
2,Jennifer Mattson,Andrea Brown Literary Agency,USA,"Picture books (lyrical, character-driven), cha...",https://andreabrownlit.com/agents/jennifer-mat...,NaN


In [ ]:
for c in ["Agent Name", "Agency", "Country"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip()

df["Agent Name"] = df["Agent Name"].str.title()
df["Agency"]     = df["Agency"].str.title()

df.head(3)


,Agent Name,Agency,Country,Genres / Categories Represented,Sources (URLs only),Notable Clients / Titles
0,Jennifer Laughran,Andrea Brown Literary Agency,USA,Children’s specialist: middle grade (contempor...,https://andreabrownlit.com/agents/jennifer-lau...,"Stephanie Perkins (ANNA AND THE FRENCH KISS), ..."
1,Jennifer March Soloway,Andrea Brown Literary Agency,USA,Children’s/YA across categories; loves high-st...,https://andreabrownlit.com/agents/jennifer-mar...,NaN
2,Jennifer Mattson,Andrea Brown Literary Agency,USA,"Picture books (lyrical, character-driven), cha...",https://andreabrownlit.com/agents/jennifer-mat...,NaN


In [ ]:
def agg_first(series):
    non_null = series.dropna()
    return non_null.iloc[0] if len(non_null) else series.iloc[0]

def agg_genres(series):
    parts = [str(x) for x in series if pd.notna(x) and str(x).strip()!=""]
    return " | ".join(parts) if parts else ""

group_cols = ["Agent Name", "Agency"]

dedup = (
    df
    .groupby(group_cols, as_index=False)
    .agg(
        **{GENRE_COL: (GENRE_COL, agg_genres)},
        **{c: (c, agg_first) for c in df.columns if c not in group_cols+[GENRE_COL]}
    )
)

print(f"Original rows: {len(df)}  →  After dedup: {len(dedup)}")
dedup[[*group_cols, GENRE_COL]].head(5)

Original rows: 885  →  After dedup: 885


,Agent Name,Agency,Genres / Categories Represented
0,A.J. Van Belle,The Booker Albert Literary Agency,Romance; contemporary; rom-com.
1,Aashna Avachat,Neighborhood Literary,YA & adult upmarket/commercial; romance; SFF.
2,Abby Saul,The Lark Group,Upmarket/commercial fiction; suspense; women’s.
3,Abi Fellows,Dhh Literary,Literary/upmarket and commercial fiction; YA c...
4,Abigail Koons,"Park, Fine & Brower Literary Management",Upmarket/commercial fiction; narrative nonfict...


In [ ]:
genre_map = {
    r"\bmg\b": "Middle Grade",
    r"\bpb\b": "Picture Book",
    r"\bya\b": "Young Adult",
    r"\bkidlit\b": "Children’s",

    r"\bya fantasy\b": "Young Adult Fantasy",
    r"\byoung adult fantasy\b": "Young Adult Fantasy",
    r"\bya contemporary\b": "Young Adult Contemporary",
    r"\byoung adult contemporary\b": "Young Adult Contemporary",

    r"\bspec fic\b": "Speculative",
    r"\bspeculative fiction\b": "Speculative",

    r"\bsci[-\s]?fi\b": "Science Fiction",
    r"\bscifi\b": "Science Fiction",
}


In [ ]:
def normalize_genre_text(text: str) -> str:
    if pd.isna(text) or not str(text).strip():
        return ""

    t = str(text).lower()

    t = re.sub(r"\(.*?\)", " ", t)

    t = t.replace("–","-").replace("—","-").replace("’","'")
    t = re.sub(r"\b(and|&|\/)\b", ",", t)
    t = re.sub(r"[;|/]", ",", t)

    t = re.sub(r"[^a-z0-9,\s\-]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()

    for patt, repl in genre_map.items():
        t = re.sub(patt, repl.lower(), t)

    toks = [x.strip() for x in t.split(",") if x.strip()]

    canon = []
    for tok in toks:
        if "middle grade" in tok and "fantasy" in tok:
            canon += ["Middle Grade", "Fantasy"]
        elif "picture book" in tok:
            canon.append("Picture Book")
        elif "young adult fantasy" in tok:
            canon.append("Young Adult Fantasy")
        elif "young adult contemporary" in tok:
            canon.append("Young Adult Contemporary")
        elif "young adult" in tok:
            canon.append("Young Adult")
        else:
            canon.append(" ".join(w.capitalize() for w in tok.split()))

    seen, out = set(), []
    for g in canon:
        if g and g not in seen:
            seen.add(g); out.append(g)

    return ", ".join(out)

dedup["Cleaned Genres"] = dedup[GENRE_COL].apply(normalize_genre_text)
dedup[["Agent Name","Agency",GENRE_COL,"Cleaned Genres"]].head(8)

,Agent Name,Agency,Genres / Categories Represented,Cleaned Genres
0,A.J. Van Belle,The Booker Albert Literary Agency,Romance; contemporary; rom-com.,"Romance, Contemporary, Rom-com"
1,Aashna Avachat,Neighborhood Literary,YA & adult upmarket/commercial; romance; SFF.,"Young Adult, Commercial, Romance, Sff"
2,Abby Saul,The Lark Group,Upmarket/commercial fiction; suspense; women’s.,"Upmarket, Commercial Fiction, Suspense, Women S"
3,Abi Fellows,Dhh Literary,Literary/upmarket and commercial fiction; YA c...,"Literary, Upmarket, Commercial Fiction, Young ..."
4,Abigail Koons,"Park, Fine & Brower Literary Management",Upmarket/commercial fiction; narrative nonfict...,"Upmarket, Commercial Fiction, Narrative Nonfic..."
5,Adam Chromy,Movable Type Management,Commercial fiction; thrillers; women’s; brand/...,"Commercial Fiction, Thrillers, Women S, Brand,..."
6,Adam Eaglin,The Cheney Agency,"Serious NF (history, politics, culture); selec...","Serious Nf, Select Fiction"
7,Adam Muhlig,"Mcintosh & Otis, Inc.","Nonfiction: science, history, culture; select ...","Nonfiction Science, History, Culture, Select F..."


In [ ]:
print("Rows with empty Cleaned Genres:", (dedup["Cleaned Genres"]=="").sum())
dedup.sample(5)[["Agent Name","Agency","Cleaned Genres"]]


Rows with empty Cleaned Genres: 202


,Agent Name,Agency,Cleaned Genres
577,Ludo Cinelli,Eve White Literary Agency,"Literary, Upmarket, Commercial Fiction, Crime,..."
278,Eric Simonoff,William Morris Endeavor,
272,Emma Leong,Janklow And Nesbit Ltd,"Upmarket, Commercial Fiction, Speculative, You..."
879,Yasmin Kane,Kane Literary Agency,"Commercial Upmarket Fiction, Young Adult, Crim..."
715,Rachel Ridout,"Harvey Klinger, Inc.","Children S, Young Adult, Illustrators, Middle ..."


In [ ]:
# Replace "Unknown" with "n/a" in the 'Cleaned Genres' column.
dedup["Cleaned Genres"] = dedup["Cleaned Genres"].replace("", "n/a")
dedup["Cleaned Genres"] = dedup["Cleaned Genres"].replace("Unknown", "n/a")

In [ ]:
(dedup["Cleaned Genres"]=="Unknown").sum()


np.int64(0)

In [ ]:
dedup[dedup["Cleaned Genres"] == "Unknown"][["Agent Name", "Agency"]].head(10)

,Agent Name,Agency


In [ ]:
out_min = dedup.drop(columns=[GENRE_COL]).rename(
    columns={"Cleaned Genres": "Genres / Categories (Cleaned)"}
)
out_min.to_excel("/agents_dedup_and_cleaned_genres.xlsx", index=False)

from google.colab import files
files.download("/agents_dedup_and_cleaned_genres.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
initial_rows = len(dedup)

# Drop rows where both 'Genres / Categories Represented' and 'Sources (URLs only)' are empty.
dedup_filtered = dedup.dropna(subset=[GENRE_COL, "Sources (URLs only)"], how='all').copy()

print(f"Original rows: {initial_rows} → After filtering: {len(dedup_filtered)}")

Original rows: 885 → After filtering: 885
